In [0]:
import sys 
!{sys.executable} -m pip install tensorflow

In [0]:
import sys 
!{sys.executable} -m pip install opencv-python


In [2]:
import os
import numpy as np
import cv2
import keras.backend as K
import random
import tensorflow as tf


def load_image(Path = './valid/XR_FINGER', size = 512):
    Images = []
    for path in Path:
        try:
            image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            Images.append(image)

        except Exception as e:
            print(str(e))

    Images = np.asarray(Images).astype('float32')
    Images=Images.reshape((len(Path),size,size,1))

    mean = np.mean(Images)			#normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    return Images

Using TensorFlow backend.


In [3]:
def load_path(root_path = './valid/XR_FINGER', size = 512):
	Path = []
	labels = []
	for root,dirs,files in os.walk(root_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
		for name in files:
			path_1 = os.path.join(root,name)
			Path.append(path_1)
			if root.split('_')[-1]=='positive':	 #positive Label is 1，otherwise 0；
				labels+=[1]   	          	 #Last level directory file patient11880\\study1_negative\\image3.png
			else:
			    labels+=[0]
	print (len(Path))
	labels = np.asarray(labels)
	return Path, labels


Path,labels=load_path(r'/content/MURA-v1.1/train/XR_FINGER',64)
images=load_image(Path,64)

5106


In [0]:
 def train_model():
        model=tf.keras.Sequential([
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu,input_shape=(64,64,1)),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=1),
        tf.keras.layers.Dropout(0.25),       
        tf.keras.layers.Conv2D(64,(3,3),padding='same',activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D((2,2),strides=2,padding='same'),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Dense(2,activation=tf.nn.softmax)
        ])
        model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    
        model.fit(images,labels,epochs=30,verbose=2)
     
        return model

In [8]:
model=train_model()

Epoch 1/30
5106/5106 - 14s - loss: 0.6258 - acc: 0.6416
Epoch 2/30
5106/5106 - 10s - loss: 0.5908 - acc: 0.6684
Epoch 3/30
5106/5106 - 10s - loss: 0.5632 - acc: 0.6931
Epoch 4/30
5106/5106 - 10s - loss: 0.5404 - acc: 0.7146
Epoch 5/30
5106/5106 - 10s - loss: 0.5302 - acc: 0.7227
Epoch 6/30
5106/5106 - 10s - loss: 0.5120 - acc: 0.7370
Epoch 7/30
5106/5106 - 10s - loss: 0.4885 - acc: 0.7479
Epoch 8/30
5106/5106 - 10s - loss: 0.4625 - acc: 0.7673
Epoch 9/30
5106/5106 - 10s - loss: 0.4296 - acc: 0.7928
Epoch 10/30
5106/5106 - 10s - loss: 0.3970 - acc: 0.8130
Epoch 11/30
5106/5106 - 10s - loss: 0.3649 - acc: 0.8310
Epoch 12/30
5106/5106 - 10s - loss: 0.3287 - acc: 0.8515
Epoch 13/30
5106/5106 - 10s - loss: 0.2883 - acc: 0.8680
Epoch 14/30
5106/5106 - 10s - loss: 0.2645 - acc: 0.8843
Epoch 15/30
5106/5106 - 10s - loss: 0.2273 - acc: 0.9070
Epoch 16/30
5106/5106 - 10s - loss: 0.2140 - acc: 0.9083
Epoch 17/30
5106/5106 - 10s - loss: 0.1941 - acc: 0.9211
Epoch 18/30
5106/5106 - 10s - loss: 0.18

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 63, 63, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 64)       

In [13]:
#Validation of test data
ValidPath,Validlabels=load_path(r'/content/MURA-v1.1/valid/XR_FINGER',64)
Validimages=load_image(ValidPath,64)

461


In [11]:
score = model.evaluate(Validimages, Validlabels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.182660516456513
Test accuracy: 0.6203905


In [0]:
model.save('finger.h5')